# CLASSIFICATION & REGRESSION MULTILAYER PERCEPTRON USING KERAS FUNCTIONAL API

_**Use Keras sequential API to build and train a multiclass image classifier model, perform classification, and then evaluate performance.**_

## Building Complex Models using the Functional API

In [33]:
# Imports required packages

from sklearn.datasets import fetch_california_housing

from sklearn.model_selection import train_test_split

import tensorflow as tf

### Loading Dataset

In [25]:
housing = fetch_california_housing()

# Prepares training and testing dataset
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Takes out validation dataset from training dataset
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, random_state=42)

### Creating Model using Functional API

In [36]:
# reset the name counters
tf.keras.backend.clear_session()

tf.random.set_seed(42)

In [39]:
# Builds the layers

input_ = tf.keras.layers.Input(shape=[8])
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

In [41]:
# Organizes the layers

normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])

2024-12-10 04:46:34.273401: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [43]:
# Shows the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 normalization (Normalization)  (None, 8)            17          ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['normalization[0][0]']          
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                              

### Training and Evaluating the model

In [46]:
# Compiles the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [48]:
# Fits the model
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
363/363 [==============================] - 2s 5ms/step - loss: 1.0630 - root_mean_squared_error: 1.0310 - val_loss: 0.6118 - val_root_mean_squared_error: 0.7822
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4522 - root_mean_squared_error: 0.6724 - val_loss: 0.4705 - val_root_mean_squared_error: 0.6859
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4057 - root_mean_squared_error: 0.6369 - val_loss: 0.8712 - val_root_mean_squared_error: 0.9334
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3823 - root_mean_squared_error: 0.6183 - val_loss: 0.4762 - val_root_mean_squared_error: 0.6901
Epoch 5/20
363/363 [==============================] - 1s 4ms/step - loss: 0.3674 - root_mean_squared_error: 0.6061 - val_loss: 0.5039 - val_root_mean_squared_error: 0.7099
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3549 - root_mean_squared_error: 0.5958 - val_loss: 0.3442 - val_

In [50]:
# Evaluates the model
mse_test = model.evaluate(X_test, y_test)
print(mse_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.3122 - root_mean_squared_error: 0.5588
[0.3122352957725525, 0.5587801933288574]


In [10]:
# Takes first three instance from test set to make predictions
X_new = X_test[:3]
y_pred = model.predict(X_new)
print(y_pred)

1/1 [==============================] - 0s 96ms/step
[[0.4887748]
 [1.2239927]
 [4.5495086]]


### Training Model to Handle Multiple Outputs
Sending different subsets of input features through the wide or deep paths.
It sends 5 features (features 0 to 4) through wide path, and 6 features (features 2 to 7) through the deep path.

In [11]:
# reset the name counters
tf.keras.backend.clear_session()

tf.random.set_seed(42)

In [12]:
# Builds the layers

input_wide = tf.keras.layers.Input(shape=[5])  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6])  # features 2 to 7

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

concat = tf.keras.layers.concatenate([norm_wide, hidden2])

output = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

In [14]:
# Compiles the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [15]:
# Trains and evaluates the model

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_val_wide, X_val_deep = X_val[:, :5], X_val[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20,
                    validation_data=((X_val_wide, X_val_deep), y_val))

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 1.5143 - root_mean_squared_error: 1.2306 - val_loss: 4.0228 - val_root_mean_squared_error: 2.0057
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4908 - root_mean_squared_error: 0.7006 - val_loss: 1.1207 - val_root_mean_squared_error: 1.0586
Epoch 3/20
363/363 [==============================] - 2s 4ms/step - loss: 0.4264 - root_mean_squared_error: 0.6530 - val_loss: 0.4734 - val_root_mean_squared_error: 0.6880
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4078 - root_mean_squared_error: 0.6386 - val_loss: 0.3755 - val_root_mean_squared_error: 0.6128
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3965 - root_mean_squared_error: 0.6297 - val_loss: 0.3766 - val_root_mean_squared_error: 0.6137
Epoch 6/20
363/363 [==============================] - 2s 4ms/step - loss: 0.3865 - root_mean_squared_error: 0.6217 - val_loss: 0.3669 - val_

In [16]:
# Evauates the model

X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
print(mse_test)

162/162 [==============================] - 1s 4ms/step - loss: 0.3280 - root_mean_squared_error: 0.5727
[0.3280080258846283, 0.5727198719978333]


### Saving and Restoring the Model

In [17]:
# Saves the model
model.save("my_MLP_regression_model.keras")

INFO:tensorflow:Assets written to: my_MLP_regression_model/assets


INFO:tensorflow:Assets written to: my_MLP_regression_model/assets


In [20]:
# Loads the model back
model = tf.keras.models.load_model("my_MLP_regression_model.keras")

In [27]:
# Makes predictions after model was loaded
y_pred = model.predict((X_test_wide, X_test_deep))

162/162 [==============================] - 0s 2ms/step


### Using Callbacks

In [28]:
# Configures the checkpoint
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints", save_weights_only=True)

# Fits the model with checkpoint callback set
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_val_wide, X_val_deep), (y_val, y_val)),
    callbacks=[checkpoint_cb])

Epoch 1/10
363/363 [==============================] - 3s 6ms/step - loss: 0.3265 - root_mean_squared_error: 0.5714 - val_loss: 0.6619 - val_root_mean_squared_error: 0.8136
Epoch 2/10
363/363 [==============================] - 2s 5ms/step - loss: 0.3308 - root_mean_squared_error: 0.5752 - val_loss: 1.0362 - val_root_mean_squared_error: 1.0179
Epoch 3/10
363/363 [==============================] - 2s 5ms/step - loss: 0.3266 - root_mean_squared_error: 0.5715 - val_loss: 0.5679 - val_root_mean_squared_error: 0.7536
Epoch 4/10
363/363 [==============================] - 2s 5ms/step - loss: 0.3236 - root_mean_squared_error: 0.5689 - val_loss: 1.1729 - val_root_mean_squared_error: 1.0830
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3259 - root_mean_squared_error: 0.5709 - val_loss: 1.5420 - val_root_mean_squared_error: 1.2418
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3292 - root_mean_squared_error: 0.5737 - val_loss: 0.7782 - val_

In [29]:
# Configures early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# Fits the model with both checkpoint and early stopping callback set
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=100,
    validation_data=((X_val_wide, X_val_deep), (y_val, y_val)),
    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3110 - root_mean_squared_error: 0.5577 - val_loss: 0.3645 - val_root_mean_squared_error: 0.6037
Epoch 2/100
363/363 [==============================] - 2s 4ms/step - loss: 0.3113 - root_mean_squared_error: 0.5579 - val_loss: 0.8973 - val_root_mean_squared_error: 0.9473
Epoch 3/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3161 - root_mean_squared_error: 0.5623 - val_loss: 0.5264 - val_root_mean_squared_error: 0.7256
Epoch 4/100
363/363 [==============================] - 2s 6ms/step - loss: 0.3127 - root_mean_squared_error: 0.5592 - val_loss: 1.1359 - val_root_mean_squared_error: 1.0658
Epoch 5/100
363/363 [==============================] - 2s 4ms/step - loss: 0.3157 - root_mean_squared_error: 0.5618 - val_loss: 1.2456 - val_root_mean_squared_error: 1.1160
Epoch 6/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3145 - root_mean_squared_error: 0.5608 - val_loss: 0.6281 

### Using TensorBoard for Visualization

In [31]:
# reset the name counters
tf.keras.backend.clear_session()

tf.random.set_seed(42)

In [32]:
# Buils model using Sequential API

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [33]:
# Compiles the model

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [38]:
# Adapts to train data by finding its feature wise mean and variance
norm_layer.adapt(X_train)

# Create log dir for TensorBoard to store logs
# Timestamp is used in naming subdirectory to seperate training experiments
log_dir = "./logs/" + datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir, profile_batch=(100, 200))

# Trains the model
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_cb])

Epoch 1/20


2024-02-12 15:19:51.569072: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-02-12 15:19:51.569151: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-02-12 15:19:51.569913: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


134/363 [==========>...................] - ETA: 0s - loss: 3.3321 - root_mean_squared_error: 1.8254

2024-02-12 15:19:52.304718: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-02-12 15:19:52.304780: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


229/363 [=================>............] - ETA: 0s - loss: 3.1019 - root_mean_squared_error: 1.7612

2024-02-12 15:19:53.122193: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-02-12 15:19:53.153610: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-02-12 15:19:53.158700: I tensorflow/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: ./logs/2024.02.12-15:19:51/plugins/profile/2024_02_12_15_19_53/pradip-Lenovo-V15-G4-AMN.xplane.pb


363/363 [==============================] - 2s 6ms/step - loss: 2.4238 - root_mean_squared_error: 1.5569 - val_loss: 1.8063 - val_root_mean_squared_error: 1.3440
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7659 - root_mean_squared_error: 0.8752 - val_loss: 0.7471 - val_root_mean_squared_error: 0.8643
Epoch 3/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6568 - root_mean_squared_error: 0.8104 - val_loss: 0.6814 - val_root_mean_squared_error: 0.8255
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6168 - root_mean_squared_error: 0.7854 - val_loss: 0.5633 - val_root_mean_squared_error: 0.7506
Epoch 5/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5861 - root_mean_squared_error: 0.7656 - val_loss: 0.5383 - val_root_mean_squared_error: 0.7337
Epoch 6/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5586 - root_mean_squared_error: 0.7474 - val_loss: 0.5096 - val_root_mean_s


The following magic command loads TensorBoard directly into the Jupyter.

[**PACKAGE REQUIREMENT:** Refer repository README to find relevant information on the installtion of Tensorboard package.]

`%load_ext tensorboard`

`%tensorboard --logdir=./logs`

TensorBoard can also be accessed through browser by executing the following command in the terminal.

`tensorboard --logdir=./logs`

## Fine-Tuning Neural Network Hyperparameters

In [148]:
# Imports required packages

import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist

from sklearn.model_selection import train_test_split

import keras_tuner as kt

#### Loading Dataset

In [151]:
# Prepares training and testing dataset

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Takes out validation dataset from training dataset
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=5000, random_state=42, stratify=y_train_full)

#### Searching for hyperparameters

In [154]:
# reset the name counters
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [156]:
def build_model(hp):
    """
    A callable that takes hyperparameters and returns a model instance
    """
    
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    
    learning_rate = hp.Float("learning_rate", min_value=1e-4, 
                             max_value=1e-2, sampling="log")
    
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Builds the model from the above hyperparameters
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", 
                  optimizer=optimizer, metrics=["accuracy"])
    
    return model

In [158]:
# Performs a basic random search for hyperparameters

random_search_tuner = kt.RandomSearch(
    build_model,                                       # Callable that takes hyperparameters and returns a Model instance
    objective="val_accuracy",                          # Metric to maximize 
    max_trials=5,                                      # Total number of trials (model configurations) to test at most
    directory="my_fashion_mnist",                      # Relative path to the working directory
    project_name="hyperparameters_random_search",   # Name to use as prefix for files saved by this Tuner.
    overwrite=True,                                    #  Overwrites tuning project files
    seed=42
)  

In [160]:
# Performs searching for best hyperparameters.
random_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 5 Complete [00h 01m 38s]
val_accuracy: 0.8532000184059143

Best val_accuracy So Far: 0.8723999857902527
Total elapsed time: 00h 10m 42s


In [162]:
# Receives top 3 models
top3_models = random_search_tuner.get_best_models(num_models=3)

best_model = top3_models[0]  # The best model

In [164]:
# Receives hyperparameters for top 3 models

top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values  # best hyperparameters values

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

In [166]:
# Gets the best trial
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8723999857902527


In [168]:
# Metrics can also be accessed directly
best_trial.metrics.get_last_value("val_accuracy")

0.8723999857902527

#### Retraining & Evaluating the Best Model

In [171]:
# Training can be continued (retraining) for a few epochs on the full training set
best_model.fit(X_train_full, y_train_full, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3597 - accuracy: 0.8694
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3523 - accuracy: 0.8721
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3445 - accuracy: 0.8751
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3388 - accuracy: 0.8768
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3358 - accuracy: 0.8773
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3318 - accuracy: 0.8789
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3286 - accuracy: 0.8809
Epoch 8/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3217 - accuracy: 0.8834
Epoch 9/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3205 - accuracy: 0.8829
Epoch 10/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0

In [173]:
# Evaluates the model on test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(test_accuracy)

313/313 [==============================] - 1s 3ms/step - loss: 0.4125 - accuracy: 0.8526
0.8525999784469604
